In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
!cp /content/drive/MyDrive/2-folder/kaggle/df_utils.py /content/
import df_utils

In [2]:
df_train_loaded = pd.read_csv('/content/drive/MyDrive/2-folder/kaggle/housing-prices-competition/train.csv')
df_test_loaded = pd.read_csv('/content/drive/MyDrive/2-folder/kaggle/housing-prices-competition/test.csv')

# num_col = len(df_train.columns)
# print(f"num_col = {num_col}")
# print(df_train["SalePrice"][:5])

df_train = df_train_loaded.drop(['Id'], axis=1)
df_test = df_test_loaded.drop(['Id'], axis=1)

# Drop all columns with with more than 20% of missing values
df_utils.drop_col_miss_val(df_train, df_test, 20)

# Fix categorical values and missing values
df_train, df_test = df_utils.prepare_df(df_train, df_test, 5, ['SalePrice'])
# print(df_train)

# Calculate the correlation of features with the target
correlation = df_train.corr()
sorted_corr = correlation['SalePrice'].sort_values(ascending=False)
# print(sorted_corr)
columns = []
for i, v in sorted_corr.items():
  if v > 0.2 and i != 'SalePrice':
    # print('index: ', i, 'value: ', v)
    columns.append(i)
# print(columns)

# Remove target label from training set
y_train = df_train['SalePrice'].values
# print(y[0:5])
df_train = df_train.drop(['SalePrice'], axis=1)

# Extract columns with high correlaton
df_train = df_train[columns]
df_test = df_test[columns]
# print(len(df_train.columns))
# print(len(df_test.columns))
# print(df_train.head(5))

# normalize dataframe
df_train=(df_train-df_train.mean())/df_train.std()
df_test=(df_test-df_test.mean())/df_test.std()

# create X and y for training
X = df_train.values
y = y_train

# create train, validation and test splits
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)